In [ ]:
import os

'''This acts as the parent class for customer types : BasicCustomer and EnterpriseCustomer.
   This stores attributes like ID,name and also methods for discount etc.
'''
class Customer:
    def __init__(self, ID, name): #initialising customer object
        self.ID = ID
        self.name = name
        self.total_money_spent = 0 #initialising total money spent to 0 at first
 
    def get_discount(self, distance_fees):
        pass

    def display_info(self):
        pass

    def get_ID(self):
        return self.ID

    def get_name(self):
        return self.name
    
    def spend_money(self, amount):
        self.total_money_spent += amount

    def get_total_money_spent(self):
        return self.total_money_spent
    
''' This class represents BasicCustomer inherited from Customer and with attributes like ID,name, distance fees calculation etc
'''
class BasicCustomer(Customer):
    
    def __init__(self, ID, name):
        super().__init__(ID, name)
        self.discount_rate = 0.10  #setting the default discount as 10%

    def get_discount(self, distance_fees):
        return self.discount_rate * distance_fees #discount

    def display_info(self):
        print("Customer Type is: Basic")
        print("Discount Rate is: {:.2%}".format(self.discount_rate))

    def set_discount_rate(self, rate):
        self.discount_rate = rate

'''This class represents EnterpriseCustomer which is also inherited from Customer. 
   This gets additional information like threshold, sets first and second discount rates.'''
class EnterpriseCustomer(Customer): #defining for Enterprise customer
    def __init__(self, ID, name, first_discount, threshold=100):
        super().__init__(ID, name)
        self.first_discount = first_discount
        self.threshold = threshold
        self.second_discount = first_discount + 0.05

    def get_discount(self, basic_fees): #calculating the discount based on the basic fee and the threshold
        if basic_fees < self.threshold:
            return self.first_discount * basic_fees
        else:
            return self.second_discount * basic_fees

    def display_info(self):
        print("Customer Type: Enterprise")
        print("First Discount Rate: {:.2%}".format(self.first_discount))
        print("Second Discount Rate: {:.2%}".format(self.second_discount))
        print("Threshold: ${}".format(self.threshold))

    def set_first_discount_rate(self, rate):
        self.first_discount = rate

'''This class represents location name along with its ID'''
class Location:
    def __init__(self, location_ID, location_name):
        self.location_ID = location_ID
        self.location_name = location_name

    def display_info(self):
        print("Location ID: {}".format(self.location_ID))
        print("Location Name: {}".format(self.location_name))

'''This class represents rates with its ID,name and also adjusts the price_per_km accordingly'''
class Rate:
    def __init__(self, rate_ID, rate_name, price_per_km):
        self.rate_ID = rate_ID
        self.rate_name = rate_name
        self.price_per_km = price_per_km

    def display_info(self): #displaying the rate information
        print("Rate ID: {}".format(self.rate_ID))
        print("Rate Name: {}".format(self.rate_name))
        print("Price per Kilometer: ${:.2f}".format(self.price_per_km))

'''This class stores information of the booking and also calculates the different fees'''
class Booking:
    def __init__(self, customer, departure, destinations, rate):
        self.customer = customer
        self.departure = departure
        self.destinations = destinations
        self.rate = rate
    '''Computes the different costs'''
    def compute_cost(self):
        total_distance_covered = sum(destination[1] for destination in self.destinations)
        basic_fees = 4.2  #setting basic fee as 4.2
        distance_fees = 0 #initialising distance fee as 0 at first
        discount = self.customer.get_discount(distance_fees)
        total_cost = basic_fees - discount #calculation of total cost without any service
        self.customer.spend_money(total_cost)
        return basic_fees, total_distance_covered, discount, total_cost

'''This class Service keeps track of information of the services availed along with
    the name,id and respective prices and also to display them. This also acts as the parent
    for the class package'''  
class Service:
    def __init__(self, service_id, service_name, service_price):
        self.service_id = service_id
        self.service_name = service_name
        self.service_price = service_price

    def get_id(self):
        return self.service_id

    def get_name(self):
        return self.service_name

    def get_price(self):
        return self.service_price

    def set_price(self, new_price):
        self.service_price = new_price

    def display_info(self):
        print("Service ID:", self.service_id)
        print("Service Name:", self.service_name)
        print("Service Price: ${:.2f}".format(self.service_price))

'''This is inherited from service and contains information of combos of services along
    with the ID, name,price and also prices are set to be 80% of the component services'''
class Package(Service):
    def __init__(self, package_id, package_name, component_services):
        super().__init__(package_id, package_name, 0)  # initializing package price to 0
        self.component_services = component_services

    def calculate_package_price(self):
        total_price = sum(service.get_price() for service in self.component_services)
        self.set_price(0.8 * total_price)  # setting package price to 80% of total component services price

    def display_info(self):
        print("Package ID:", self.get_id())
        print("Package Name:", self.get_name())
        print("Package Price: ${:.2f}".format(self.get_price()))
        print("Component Services:")
        for service in self.component_services:
            print("  -", service.get_name())

'''As the name suggests, this class contains records of already existing customers, locations, rates, services and bookings from the .txt files and also
    engages in displaying them whenever needed. This also holds the fucntions to check the availability of the files along with reading the same.'''
class Records:
    def __init__(self):
        self.customers = []  # creating lists to store customer, location, rate, service and bookings
        self.locations = [] 
        self.rates = [] 
        self.services = []  
        self.bookings = []
        
    def read_customers(self, file_name):
        try:
            with open("customers.txt", "r") as file: #reading "customers.txt"
                for line in file:
                    data = line.strip().split(',')
                    cust_id = int(data[0])
                    cust_name = data[1].strip()
                    customer_type = data[2].strip()
                    if customer_type == 'B':
                        discount_rate = float(data[3])
                        customer = BasicCustomer(cust_id, cust_name)
                        customer.set_discount_rate(discount_rate)
                    elif customer_type == 'E':
                        first_discount = float(data[3])
                        threshold = int(data[4])
                        customer = EnterpriseCustomer(cust_id, cust_name, first_discount, threshold)
                    else:
                        continue
                    self.customers.append(customer)
        except FileNotFoundError:
            print(f"Error: {file_name} not found")

    def read_locations(self, file_name):
        try:
            with open("locations.txt", 'r') as file: #reading "locations.txt"
                for line in file:
                    data = line.strip().split(',')
                    location = Location(data[0].strip(), data[1].strip())
                    self.locations.append(location)
        except FileNotFoundError:
            print(f"Error: {file_name} not found")

    def read_rates(self, file_name):
        try:
            with open("rates.txt", 'r') as file: #reading "rates.txt"
                for line in file:
                    data = line.strip().split(',')
                    rate = Rate(data[0].strip(), data[1].strip(), float(data[2]))
                    self.rates.append(rate)
        except FileNotFoundError:
            print(f"Error: {file_name} not found")

    def read_services(self, file_name):
        try:
            with open("services.txt", 'r') as file: #reading "services.txt"
                for line in file:
                    data = line.strip().split(', ')
                    service_id = data[0].strip()
                    service_name = data[1].strip()

                    if service_id.startswith("S"):
                        service_price = float(data[2])
                        service = Service(service_id, service_name, service_price)
                        self.services.append(service)
                    elif service_id.startswith("P"):
                        component_ids = data[2:]
                        component_services = []
                        for component_id in component_ids:
                            component_service = self.find_service(component_id)
                            if component_service:
                                component_services.append(component_service)
                            else:
                                print(f"Warning: Component service with ID '{component_id}' not found for package '{service_id}'.")
                        package = Package(service_id, service_name, component_services)
                        package.calculate_package_price()
                        self.services.append(package)
        except FileNotFoundError:
            print(f"Error: {file_name} not found")
            
    def read_bookings(self, file_name):
        try:
            with open("bookings.txt", 'r') as file: #reading "bookings.txt"
                for line in file:
                    data = line.strip().split(', ')
                    booking_info = {
                        "customer": self.find_customer(data[0].strip()),
                        "departure": self.find_location(data[1].strip()),
                        "destinations": [],
                        "rate": self.find_rate(data[-2].strip()),
                        "service": self.find_service(data[-1].strip())
                    }
                    for i in range(2, len(data) - 2, 2):
                        destination_location = self.find_location(data[i + 1].strip())
                        if destination_location:
                            booking_info["destinations"].append((destination_location, float(data[i])))
                    self.bookings.append(booking_info)
        except FileNotFoundError:
            print("Cannot load the booking file")

    def find_customer(self, search_value):
        for customer in self.customers:
            if str(customer.get_ID()) == search_value or customer.get_name() == search_value:
                return customer
        return None

    def find_location(self, search_value):
        for location in self.locations:
            if location.location_ID == search_value or location.location_name == search_value:
                return location
        return None

    def find_rate(self, search_value):
        for rate in self.rates:
            if rate.rate_ID == search_value or rate.rate_name == search_value:
                return rate
        return None

    def find_service(self, search_value):
        for service in self.services:
            if service.get_id() == search_value or (search_value.startswith('P') and service.get_id() == search_value[1:]):
                return service
        return None

    def list_customers(self):
        for customer in self.customers:
            print("Customer ID:", customer.get_ID())
            print("Customer Name:", customer.get_name())
            if isinstance(customer, BasicCustomer):
                print("Customer Type: Basic")
                print("Discount Rate: {:.2%}".format(customer.discount_rate))
            elif isinstance(customer, EnterpriseCustomer):
                print("Customer Type: Enterprise")
                print("First Discount Rate: {:.2%}".format(customer.first_discount))
                print("Second Discount Rate: {:.2%}".format(customer.second_discount))
                print("Threshold: ${}".format(customer.threshold))
            print()

    def list_locations(self):
        for location in self.locations:
            location.display_info()
            print()

    def list_rates(self):
        for rate in self.rates:
            rate.display_info()
            print()

    def list_services(self):
        for service in self.services:
            service.display_info()
            print()

    def check_files_exist(self): #checking whether the files are present
        required_files = ["customers.txt", "locations.txt", "rates.txt", "services.txt","bookings.txt"]
        for file_name in required_files:
            if not os.path.exists(file_name):
                print(f"Error: {file_name} is missing in the local directory.")
                return False
        return True

    def read_all_files(self):
        if self.check_files_exist():
            self.read_customers("customers.txt")
            self.read_locations("locations.txt")
            self.read_rates("rates.txt")
            self.read_services("services.txt")
            self.read_bookings("bookings.txt")
            
    def update_basic_customer_discount(self, new_disc_rates):
        for customer in self.customers:
            if isinstance(customer, BasicCustomer):
                customer.set_discount_rate(new_disc_rates)
                
    
    def display_all_bookings(self):
        if not self.bookings:
            print("No bookings found.")
            return

        print("\nBookings:")
        print("---------------------------------------------------------")
        for booking_info in self.bookings:
            customer = booking_info["customer"]
            departure = booking_info["departure"].location_name
            destinations = booking_info["destinations"]
            rate = booking_info["rate"]
            service = booking_info["service"]

            if rate is None:
                print("Error: Rate not found for this booking. Skipping this booking.")
                continue

            total_distance_covered = sum(distance for _, distance in destinations)
            basic_fees = total_distance_covered * rate.price_per_km
            discount = customer.get_discount(basic_fees)

            print(f"Customer Name: {customer.get_name()}")
            print(f"Departure Location: {departure}")
            print("Destinations:")
            for destination, distance in destinations:
                print(f"  - {destination.location_name} ({distance:.2f} km)")
            print(f"Rate Type: {rate.rate_name} (AUD per km: ${rate.price_per_km:.2f})")
            print(f"Total Distance: {total_distance_covered:.2f} km")
            print("---------------------------------------------------------")
            print(f"Basic Fee: ${basic_fees:.2f}")
            if service.get_id().startswith('P'):
                print(f"Service/Package: {service.get_name()} (Price: ${service.get_price():.2f})")
            if discount > 0:
                print(f"Discount: ${discount:.2f}")
            total_cost = basic_fees - discount
            print("---------------------------------------------------------")
            print(f"Total Cost: ${total_cost:.2f}")
            print("---------------------------------------------------------")

       
    def find_most_popular_customer(self):
            if not self.customers:
                return None

            max_money_spent = max(customer.get_total_money_spent() for customer in self.customers)
            popular_customers = [customer for customer in self.customers if customer.get_total_money_spent() == max_money_spent]
            return popular_customers

    def display_most_popular_customer(self):
            popular_customers = self.find_most_popular_customer()
            if not popular_customers:
                print("No customers found.")
                return

            print("\nMost Popular Customer(s)")
            print("---------------------------------------------------------")
            for customer in popular_customers:
                print(f"Customer Name: {customer.get_name()}")
                print(f"Total Money Spent: ${customer.get_total_money_spent():.2f}")
                print("---------------------------------------------------------")

'''This is the main class of the program.
  There are several options available on a user-friendly menu, including booking trips, customer data, destinations, pricing and more. 
  By choosing options from the menu, users can interact with the programme.'''
class Operations:
    def __init__(self):
        self.records = Records()

    def start(self):
        self.records.read_all_files()

        while True:
            print("########## Welcome to RMIT taxi booking menu ##########")
            print("1. Book a trip")
            print("2. Display existing customers")
            print("3. Display existing locations")
            print("4. Display existing rate types")
            print("5. Display existing services/packages")
            print("6. Add new locations")
            print("7. Add or update rate types")
            print("8. Adjust the discount rate of all Basic customers")
            print("9. Adjust the discount rate of an Enterprise customer")
            print("10. Display all bookings")
            print("11. Display the most valuable customer")
            print("12. Display customer booking history")
            print("13. Save the files")
            print("14. Exit the program")

            option = input("Enter your choice (1/2/3/4/5/6/7/8/9/10/11/12/13/14): ")

            if option == "1":
                self.book_trip()
            elif option == "2":
                self.records.list_customers()
            elif option == "3":
                self.records.list_locations()
            elif option == "4":
                self.records.list_rates()
            elif option == "5":
                self.records.list_services()
            elif option == "6":
                self.add_new_locations()
            elif option == "7":   
                self.add_or_update_rate_types_and_prices()
            elif option == "8":
                self.adjust_basic_customer_discount()
            elif option == "9":    
                self.adjust_enterprise_discount()
            elif option == "10":
                self.records.display_all_bookings()
            elif option == "11":
                self.records.display_most_popular_customer()
            elif option == "12":
                self.display_customer_booking_history()
            elif option == "13":
                self.save_data()
            elif option == "14":
                print("Exiting the taxi service. Thank you!")
                break
            else:
                print("Invalid choice. Please select a valid option from 1 to 14.")

    '''When a user enters a new location, the method first takes that input, turns it into a list of locations, and then iterates through this list. 
    It checks to see if each location has previously been recorded. 
    If not, it creates a new Location object, adds it to the list of location, and generates a unique ID. '''          
    def add_new_locations(self):
        new_locations = input("Enter new locations separated by commas: ")
        new_location_list = [location.strip() for location in new_locations.split(",")]

        extra_location = [] #an empty list to add the new locations

        for new_location in new_location_list:
            if new_location in [location.location_name for location in self.records.locations]:
                print(f"{new_location} is already an existing location.")
            else:
                new_location_id = f"L{len(self.records.locations) + 1}" 
                location = Location(new_location_id, new_location) 
                self.records.locations.append(location)
                extra_location.append(location) 

        if extra_location:
            print("Updated locations:")
            for location in extra_location:
                location.display_info()
        else:
            print("No new locations were added.")
            
    def adjust_basic_customer_discount(self):
        while True: 
            new_discount_rate_input = input("Enter the new discount rate for Basic customers (in percentage): ")
            try:
                new_disc_rates = float(new_discount_rate_input)
                if new_disc_rates <= 0:
                    print("Discount rate must be a positive number. Please enter a valid rate.")
                    continue
                self.records.update_basic_customer_discount(new_disc_rates)
                print("Discount rate for all Basic customers updated successfully to: ",new_disc_rates)
                break 
            except ValueError:
                print("Not valid. Please enter a valid discount rate (a positive number).")
                
    def adjust_enterprise_discount(self):
        while True:
            customer_input = input("Enter the name or ID of an Enterprise customer: ")
            customer = self.records.find_customer(customer_input)

            if customer is None or not isinstance(customer, EnterpriseCustomer):
                print("Invalid Enterprise customer. Please enter a valid Enterprise customer name or ID.")
                continue

            try:
                new_disc_rates = float(input("Enter the new first discount rate (e.g., 0.2 for 20%): "))

                if new_disc_rates <= 0:
                    print("Discount rate must be a positive number. Please enter a valid rate.")
                    continue

                customer.set_first_discount_rate(new_disc_rates)
                print(f"Discount rate for {customer.get_name()} updated to {new_disc_rates * 100}%")
                break

            except ValueError:
                print("Invalid input. Please enter a valid discount rate (a positive number).")
                
    '''By gathering customer information like ID, name, and type (Basic or Enterprise), the book_trip function makes it easier to reserve a taxi ride. 
    It directs the choice of the departure and destination(s) as well as their corresponding distances. 
    Additionally, it enables the choice of a travel cost type and the ability to add on additional services or packages. 
    The final taxi charge is also calculated, taking into account basic fees, distance fees, discountss, and service fees if applied. 
    It provides a  receipt showing all trip information, including cost breakdown.'''
    
    def book_trip(self):
        
        destinations = [] #creating an empty list to store the destination(s)
        while True:
            cust_id = input("Enter customer ID: ")

            if not cust_id.isdigit():
                print("Invalid customer ID. Please enter a valid number for customer ID.")
                continue

            customer = self.records.find_customer(cust_id)

            if customer is not None:
                print("Customer ID already in use. Enter a unique customer ID.")
                continue

            cust_name = input("Enter customer name: ")

            while True:
                if cust_name.isalpha():
                    break
                else:
                    print("This is not a valid name, please enter a valid name! ")
                    cust_name = input("Enter customer name: ")

            customer_type = input("Enter customer type (Basic/Enterprise): ")

            if customer_type == "Basic":
                customer = BasicCustomer(cust_id, cust_name)
                discount_rate = float(input("Enter discount rate (in percentage): ")) / 100
                customer.set_discount_rate(discount_rate)
            elif customer_type == "Enterprise":
                first_discount = float(input("Enter the first discount rate (in percentage): ")) / 100
                threshold = int(input("Enter the threshold: "))
                customer = EnterpriseCustomer(cust_id, cust_name, first_discount, threshold)
            else:
                print("Invalid customer type. Booking canceled.")
                return

            self.records.customers.append(customer)
            break

        departure = input("Enter departure location: ")
        departure_location = self.records.find_location(departure)

        while departure_location is None:
            print("Departure location not found. Please enter a valid location.")
            departure = input("Enter departure location: ")
            departure_location = self.records.find_location(departure)

        destinations = []

        while True:
            destination = input("Enter destination location (type 'done' to finish entering destinations): ")

            if destination == "done":
                if not destinations:
                    print("No destinations entered. Please enter at least one destination.")
                    continue
                break

            if destination == departure:
                print("Destination and Departure are the same. Enter a different destination.")
                continue

            if any(d[0] == destination for d in destinations): #checks if the entered destination is already enetered
                print("Destination already entered. Enter a different destination.")
                continue

            destination_location = self.records.find_location(destination)

            while destination_location is None:
                print("Destination location not found. Please enter a valid location.")
                destination = input("Enter destination location: ")
                destination_location = self.records.find_location(destination)

            while True:
                distance_input = input("Enter distance from departure to {} in kilometers: ".format(destination))
                if distance_input.replace(".", "").isdigit():
                    distance = float(distance_input)
                    if distance <= 0:
                        print("Distance must be greater than zero. Please enter a valid distance.")
                        continue
                    destinations.append((destination, distance))
                    break
                else:
                    print("Invalid input. Please enter a valid number for distance.")
                    continue


        while True:
            rate_type = input("Enter rate type: ")
            rate = self.records.find_rate(rate_type)
            if rate is not None:
                break
            else:
                print("Rate type not found. Please enter a valid rate type.")
                continue
                
        

        service_fees = 0 #setting service fee as 0 initially

        while True:
            order_service = input("Do you want to order an extra service/package? (y/n): ")

            if order_service.lower() == 'y':
                print("Available services/packages:")
                self.records.list_services()
                service_name = input("Enter the name of the service/package you want to order: ")
                selected_service = self.records.find_service(service_name)
                if selected_service is not None:
                    service_fees += selected_service.get_price()
                    print(f"{selected_service.get_name()} added to the booking.")
                else:
                    print("Service/package not found. Please enter a valid service/package name.")
            elif order_service.lower() == 'n':
                break
            else:
                print("Invalid input. Please enter 'y' for yes or 'n' for no.")
                continue
        
        # Taxi fare calculation:
        basic_fees = 4.2 
        total_distance_covered = sum(distance for _, distance in destinations)
        distance_fees = rate.price_per_km * total_distance_covered
        total_discount = customer.get_discount(distance_fees)
        total_cost = basic_fees + distance_fees - total_discount + service_fees #total cost calculation
        
        ############ PRINTING THE RECIEPT ##############

        print("\nTaxi Receipt")
        print("---------------------------------------------------------")
        print(f"Name: {cust_name}")
        print(f"Departure: {departure}")
        print("Destinations:")
        for destination, distance in destinations:
            print(f"  - {destination} ({distance:.2f} km)")
        print(f"Rate: {rate.rate_name} (AUD per km: ${rate.price_per_km:.2f})")
        print(f"Distance: {total_distance_covered:.2f} km")
        print("---------------------------------------------------------")
        print(f"Basic fee: ${basic_fees:.2f} (AUD)")
        print(f"Distance fee: ${distance_fees:.2f} (AUD)")
        if service_fees > 0:
            print(f"Service/package fee: ${service_fees:.2f} (AUD)")
        print(f"Discount: ${total_discount:.2f} (AUD)")
        print("---------------------------------------------------------") 
        print(f"Total cost: ${total_cost:.2f} (AUD)")
        print("****************** Have a good trip ********************")
        
    '''Users can either add new rate types or update existing ones by using the method add_or_update_rate_types_and_prices. 
    When adding, it prompts the user to enter the price for each rate type and then adds the additional rate types after checking the input. 
    When updating, it uses the most recent rate types and their prices, verifies the input, and then adjusts current rates. 
    This also handles any input mistakes.'''

    def add_or_update_rate_types_and_prices(self): 
        pref = input("Do you want to add/update rate type(s)? Enter either add or update: ").lower() # Asking the user to whether add or update the rates
        print("You have chosen to ",pref)
        if pref == 'add': #If add, add_rate_types() is called
            self.add_rate_types()
        elif pref == 'update': #If update, update_rate_types() is called
            self.update_rate_types()
        else:
            print("Invalid choice. Please enter 'add' or 'update'.")

    def add_rate_types(self):
        rate_types = input("Enter rate types separated by commas: ").strip().split(',')
        prices_input = input("Enter corresponding prices separated by commas: ").strip().split(',')

        rate_types = [rt.strip() for rt in rate_types]
        prices_input = [p.strip() for p in prices_input]
        
        if len(rate_types) != len(prices_input): #to check whether the number of rate types and their respective number of rates match
            print("Error: The number of rate types and prices must match.")
            return

        prices = [] #empty list to store the entered prices
        for price_str in prices_input:
            try:
                price = float(price_str)
                if price <= 0:
                    print("Error: Prices must be positive numbers.")
                    return
                prices.append(price)
            except ValueError:
                print("Error! Please enter valid numbers.")
                return

        
        for rate_type, price in zip(rate_types, prices): #the zip here is used to update the price for each type based on the corresponding price in the prices list
            new_rate = Rate(f"R{len(self.records.rates) + 1}", rate_type, price)
            self.records.rates.append(new_rate)
            print(f"Added new rate type: {rate_type} and Price: ${price:.2f}")

        print("The rate types and their prices have been added successfully.")

    def update_rate_types(self):
        rate_types = input("Enter rate types to update separated by commas: ").strip().split(',')
        prices_input = input("Enter corresponding updated prices separated by commas: ").strip().split(',')

        
        rate_types = [r.strip() for r in rate_types]
        prices_input = [p.strip() for p in prices_input]

        
        if len(rate_types) != len(prices_input):
            print("Error! The number of rate types and prices must match.")
            return

        
        prices = [] #empty list to store prices
        for price_str in prices_input:
            try:
                price = float(price_str)
                if price <= 0:
                    print("Error! Prices must be in positive numbers.")
                    return
                prices.append(price)
            except ValueError:
                print("Error: Invalid price format. Please enter valid number.")
                return
            
        for rate_type, price in zip(rate_types, prices): # here zip is used to update the price for each type based on the corresponding price in the prices list
            existing_rate = self.records.find_rate(rate_type)
            if existing_rate:
                existing_rate.price_per_km = price
                print(f"Updated rate type: {rate_type} - New Price: ${price:.2f}")
                print("The rate types and their respective prices updated successfully.")
            else:
                print(f"Rate type '{rate_type}' not found. Skipping update.")

        
        
    '''When a customer's name is entered, this function obtains their booking history and displays information about their departure, 
      destinations, services, and overall cost. 
      It iterats through the bookings and displays  details for the chosen customer or prompts the user if no booking history is found.'''
    def display_customer_booking_history(self):
        while True:
            cust_name = input("Please enter customer name: ")
            customer = self.records.find_customer(cust_name)

            if customer:
                print(f"Booking history of {cust_name}:")

                
                cust_bookings = [booking for booking in self.records.bookings if booking["customer"] == customer]

                if not cust_bookings:
                    print("No booking history found for this customer.")

                else:
                    
                    for i, booking in enumerate(cust_bookings, start=1): #iterating through cust_bookings; start is set to 1 to start from 1 instead of 0
                        departure = booking["departure"].location_name
                        destinations = [destination[0].location_name for destination in booking["destinations"]]
                        service = booking["service"]
                        
                        
                        total_cost = booking.get("total_cost", 0)

                        if service:
                            service_name = service.get_name()
                        else:
                            service_name = "No service available."

                        print(f"Booking {i}")
                        print(f"Departure: {departure}")
                        print(f"Destination: {', '.join(destinations)}")
                        print(f"Service: {service_name}")
                        print(f"Total cost: {total_cost:.2f}")

                break
            else:
                print("Customer name is invalid. Please enter a valid customer name.")

    '''The save_data function acts as parent for the rest and is in charge of saving the various data elements, including customers, locations, rates, and bookings, to the appropriate text files. 
    The write_ functions are each used to write a particular datato the related file.'''
    def save_data(self):
        self.write_customers()
        self.write_locations()
        self.write_rates()
        self.write_bookings()
 
    def write_customers(self):
        with open('customers.txt', 'w') as file:
            for customer in self.records.customers:
                
                file.write(f"{customer.get_ID()}, {customer.get_name()}, {self.get_customer_type(customer)}, {self.get_customer_data(customer)}\n")

    def write_locations(self):
        with open('locations.txt', 'w') as file:
            for location in self.records.locations:
                
                file.write(f"{location.location_ID}, {location.location_name}\n")

    def write_rates(self):
        with open('rates.txt', 'w') as file:
            for rate in self.records.rates:
                
                file.write(f"{rate.rate_ID}, {rate.rate_name}, {rate.price_per_km}\n")

    def write_bookings(self):
        with open('bookings.txt', 'w') as file:
            for booking in self.records.bookings:
                
                file.write(f"{booking.serialize()}\n")

'''This is an important block of code for the entire program to function as it ensures data initialisation, separating data management
   in Records and Operations. This also ensures the smooth flow of the entire program '''
if __name__ == "__main__":
    records = Records()
    records.read_all_files()
    operations = Operations()
    operations.start()


''' The taxi booking program follows an object-oriented approach to model and manage various entities and their interactions within the taxi service. 
It exhibits several key principles of object-oriented programming, such as encapsulation, inheritance, and polymorphism, to achieve modularity and maintainability. 
Entities like Customers, Locations, and Rates are represented as classes, encapsulating their attributes and behaviors.

Inheritance and Polymorphism:

The use of inheritance allows for performing operations like getting customer types (BasicCustomer, EnterpriseCustomer) to inherit from a base Customer class. This
is also used in other parts of the code This promotes code reuse.
Polymorphism is employed as different customer types can be used interchangeably when making bookings.

Static Variables and Methods:

Static variables are used to generate consecutive IDs for newly created customers and rates. This ensures that the counter is shared across all objects of the class.
Static variables are also used for the threshold of EnterpriseCustomer, maintaining a common value among all instances.


'while' loop has been used commonly in this program because, in many placesm users are given another chance if they enter a wrong value
 or should be able to choose another option. For this function, a while loop is the most preferred loop.

References:

1) RMIT - Programming Fundamentals - Modules 1 to 10.
2) The python tutorial (no date) Python documentation. Available at: https://docs.python.org/3/tutorial/index.html (Accessed: 15 October 2023). 
3) (No date) Python OS module. Available at: https://www.w3schools.com/python/module_os.asp (Accessed: 15 October 2023). 
4) (No date a) Python inheritance. Available at: https://www.w3schools.com/python/python_inheritance.asp (Accessed: 15 October 2023). 
5) Zip() in Python (2023) GeeksforGeeks. Available at: https://www.geeksforgeeks.org/zip-in-python/ (Accessed: 15 October 2023). 


'''



########## Welcome to RMIT taxi booking menu ##########
1. Book a trip
2. Display existing customers
3. Display existing locations
4. Display existing rate types
5. Display existing services/packages
6. Add new locations
7. Add or update rate types
8. Adjust the discount rate of all Basic customers
9. Adjust the discount rate of an Enterprise customer
10. Display all bookings
11. Display the most valuable customer
12. Display customer booking history
13. Save the files
14. Exit the program


Enter your choice (1/2/3/4/5/6/7/8/9/10/11/12/13/14):  1
Enter customer ID:  123
Enter customer name:  sdfsd
Enter customer type (Basic/Enterprise):  Basic
Enter discount rate (in percentage):  22
Enter departure location:  Chadstone
Enter destination location (type 'done' to finish entering destinations):  done


No destinations entered. Please enter at least one destination.


Enter destination location (type 'done' to finish entering destinations):  Fitzroy
Enter distance from departure to Fitzroy in kilometers:  11
Enter destination location (type 'done' to finish entering destinations):  done
Enter rate type:  peak
Do you want to order an extra service/package? (y/n):  n



Taxi Receipt
---------------------------------------------------------
Name: sdfsd
Departure: Chadstone
Destinations:
  - Fitzroy (11.00 km)
Rate: peak (AUD per km: $1.80)
Distance: 11.00 km
---------------------------------------------------------
Basic fee: $4.20 (AUD)
Distance fee: $19.80 (AUD)
Discount: $4.36 (AUD)
---------------------------------------------------------
Total cost: $19.64 (AUD)
****************** Have a good trip ********************
########## Welcome to RMIT taxi booking menu ##########
1. Book a trip
2. Display existing customers
3. Display existing locations
4. Display existing rate types
5. Display existing services/packages
6. Add new locations
7. Add or update rate types
8. Adjust the discount rate of all Basic customers
9. Adjust the discount rate of an Enterprise customer
10. Display all bookings
11. Display the most valuable customer
12. Display customer booking history
13. Save the files
14. Exit the program
